### Imports

In [4]:
import os
import json
import numpy as np

### Parse the ZNE data

The raw experimental are available at `experimental data/data`. The following code parses the ZNE data:

In [15]:
# Folder containing your JSON files
folder_path = "experimental data/data"

# List to store the data from the filtered JSON files
all_data = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".json") and "ZNE" in filename:  # Check if the file is a JSON file and contains "ZNE"
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r") as file:
            data = json.load(file)  # Parse the JSON file
            all_data.append(data)  # Add the parsed data to the list

### Parsing other useful data

To extract other useful data such as exact minimum eigen values, estimated minimum eigen values etc, the VQE data needs to be parsed.

In [ ]:
# List to store the data from the filtered JSON files
vqe_data = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".json") and "VQE" in filename:  # Check if the file is a JSON file and contains "ZNE"
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r") as file:
            data = json.load(file)  # Parse the JSON file
            vqe_data.append(data)  # Add the parsed data to the list

### Mean and standard deviation of estimated minimum energy eigen values

In [21]:
eigen_vals = vqe_data[0]["Optimized_minimum_cost"]
eigen_vals_mean = np.mean(eigen_vals)
print(f"Mean: {eigen_vals_mean}")

Mean: -8.31556574652772


In [20]:
# Standard deviation in eigenvalues
eigen_vals_sd = np.std(eigen_vals)
print(f"Standard deviation: {eigen_vals_sd}")

Standard deviation: 0.131728374668213


### Mean and standard deviation of ZNE values

In [24]:
zne_vals = []
for data in all_data:
    zne_val = data["ZNE_values"]["Value"]
    zne_vals.append(zne_val)

zne_mean = np.mean(zne_vals)
print(f"Mean ZNE: {zne_mean}")


Mean ZNE: -8.371059728319427


In [25]:
zne_sd = np.std(zne_vals)
print(f"ZNE standard deviation: {zne_sd}")

ZNE standard deviation: 0.12800737993647104
